## <span style="color:Orange"> **Import Packages and Settings** </span>

In [1]:
import json
import pandas as pd
import os
import re
import numpy as np
from sklearn import naive_bayes

from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from nltk import bigrams

import sklearn.model_selection as ms
import random

import matplotlib.pyplot as plt
import matplotlib as matplotlib
import os

from pylab import figure, axes, pie, title, show

import os.path as path
import sys
from pathlib import Path

## <span style="color:Orange"> **Import Data** </span>

In [2]:
data = pd.read_json('../train_data/merchantSwipeDump_old.json')

## <span style="color:Orange"> **Import Data Cleaning Module and Clean** </span>

In [3]:
import os.path as path
import sys
from pathlib import Path

#cwd = os.getcwd()
#two_up =  Path(path.abspath(os.getcwd())).parents[1]
#os.chdir(two_up)

sys.path.insert(0, '../../utility_files')

#sys.path.insert(0,a)# './utilitiy_files/cleanData')
import cleanData



cleanData.clean_help()

remove_string = '[]' #'^[0-9]*[0-9]$|^www$|^com$|^ave|^street$|^st$|^road$|^and$|^inc$|^at$|^drive$|^of$|^main$|^the$|^[ewns]$|^#|^[0-9]*th$|^[0-9]*rd$|^1st$|^store$|^south$'
split_string = "[- ./*']"

cleanData.clean(data,rejoin_col_strings_bi=False,lowercase=True,split_string=split_string,remove_string = remove_string,join_mcc_bi=True) 


The inputs are (df=data, old_col="merchant_string", col="merchant_string_clean",
    split_string=split_string, remove_string = remove_string,
    lowercase=True, remove_empty_strings_bi=True, join_mcc_bi=True,
    rejoin_col_strings_bi=True')

    REQUIRES the mcc column in df is called "mcc"

    df: is the data frame
    old_col: is the name of the column to clean - must be in the df
    col: is the name of the new col - need not be in the df'
    split_string: takes a regex to split string on to remove the split symbol and create divides in the string
    remove_string: takes a regex and removes everything that matches the regex
    join_mcc: joins the mcc column to the col selcted if mcc is not nan - mcc must be labeled "mcc"
    rejoin_col_strings_bi: set to true if you want a the cleaned column to be a string False will be a list


In [4]:
brands = sorted(list(set(data.mapped_brand.astype(str))))+['greatclips'] #added brand

data['temp1'] = data['merchant_string_clean'].apply(lambda L: ''.join(L))
data['temp2'] = data['temp1'].apply(lambda s: list(x if s.find(x)!=-1 else '' for x in list(brands)))
data['temp2'] = data['temp2'].apply(lambda s: ''.join(s))

L =[]
for i in range(len(data)):
    if data['temp2'][i]== '':
        L.append(data['merchant_string_clean'][i])
    else:
        L.append([data['temp2'][i]]+data['merchant_string_clean'][i])


data['merchant_string_clean'] = list(map(lambda s: ' '.join(s), L))

## <span style="color:Orange"> **Fill mapped_brands with the dictionary** </span>

In [5]:
#------------- Run Dictionary --------------#

## Import your packages

## Functions
# Change mcc to string
def col_to_str(df, col):
    df[col] = df[col].apply(lambda x: str(x))   

# Pre-process
def preprocess(df):
    ## Remove ".0" from MCC column
    df['mcc']=df['mcc'].apply(lambda x: str(x).strip(".0"))
    
    ## Change mcc col
    col_to_str(data, 'mcc')
    
    ## Change mcc nan to NaN
    df['mcc'] = np.where(df['mcc']=='nan', np.nan, df['mcc'])

# Add a new column (to then run clean on)
def new_merchant_col(df, col_old, col_new):
    df[col_new] = data[col_old]
    
# Make a column lowercase
def lowercase_col(df, col):
    df[col]=df[col].apply(lambda x: x.lower())

#----
    
## Import data and run preprocess
# Read in data
#data = pd.read_json('merchantSwipeDump_old.json')


# Drop Unnamed:0 column
#data = data.drop("Unnamed: 0", axis=1)

# Preprocess (change mcc to string and nan to NaN, remove mccs' ".0")
preprocess(data)

#----

## Create new columns: mapped_brand_dict column (lowercase merchant_string) and blank one to predict into
new_merchant_col(data, 'merchant_string_clean', 'merchant_string_dict') # so not cleaned
lowercase_col(data, 'merchant_string_dict') # dictionary below needs lowercase to work
data['mapped_brand_dict_3'] = data['mapped_brand']
#data.head()

#----
## Dictionary

# Map merchant strings
mapping_dict = {'vend': 'vendingmachine', #'usa': 'vendingmachine',
                'wal-mart': 'walmart', 'walmart': 'walmart', 'wm supercenter': 'walmart',
                'uber ': 'uber', # or uber
                'paypal': 'paypal',
                "mcdonald's": 'mcdonalds', 'mcdonalds': 'mcdonalds',
                'target t-': 'target', 'target.com': 'target',
                'cvs/pharm': 'cvs',
                'walgreens': 'walgreens',
                'starbucks': 'starbucks', 
                'chick-fil-a': 'chickfila',
                'gamestop': 'gamestop', 
                'google \*': 'googleplay', # or google,
                'kroger': 'kroger',
                'chipotle': 'chipotle',
                'apl\* itunes.com/bill': 'appleitunes', # needs a \ before *
                'dunkin': 'dunkindonuts', 
                'amazon': 'amazon',
                'lyft': 'lyft',
                '7-eleven': 'seveneleven', '7 eleven': 'seveneleven',
                "victoria's secret": 'victoriassecret', 'victoriassecret.com':'victoriassecret',
                'etsy.com': 'etsy', 'etsy': 'etsy',
                'duane reade': 'duanereade',
                'taco bell': 'tacobell',
                'dollar-general': 'dollargeneral', 'dollar general': 'dollargeneral', 'dollar ge': 'dollargeneral',
                "wendy's": 'wendys', 'wendys': 'wendys',
                'amc ': 'amc',
                'safeway store': 'safeway', 'safeway': 'safeway',
                'panera bread': 'panerabread',
                'subway restaurant': 'subway',
                'sonic': 'sonic',
                'rite aid store': 'riteaidpharmacy',
                'chevron/': 'chevron',
                'forever 21': 'forever21',
                'dollar tr': 'dollartree',
                "claire's": 'claires',
                'dairy queen': 'dairyqueen',
                "sq \*tomy's": 'tomys', # needs a \ before *
                'qt ': 'quiktrip',
                'microsoft ': 'microsoft',
                'ulta.com': 'ultabeauty', 'ulta #': 'ultabeauty',
                'playstation network': 'playstation',
                'barnes an': 'barnesandnoble', 'barnes & noble': 'barnesandnoble', 'barnesnob': 'barnesandnoble',
                'burger king': 'burgerking',
                'riotgam\*': 'riotgames',
                'michaels stores': 'michaels',
                'sephora': 'sephora',
                'five guys': 'fiveguys', '5guys': 'fiveguys',
                'five below': 'fivebelow',
                'bath and body works': 'bathandbodyworks', 'bath & body works' : 'bathandbodyworks',
                'shake shack': 'shakeshack',
                'chopt': 'chopt',
                'urban-out': 'urbanoutfitters', 'urban out': 'urbanoutfitters',
                "domino's": 'dominos',
                'regal cinemas': 'regalcinemas', 'edwards':'regalcinemas',
                'circle k': 'circlek',
                'sweetgreen': 'sweetgreen',
                'wholefds': 'wholefoods',
                'coca cola': 'cocacola', 'coca-cola': 'cocacola',
                'nyctaxi': 'nyctaxi', 'nyc taxi': 'nyctaxi',
                'shell': 'shell',
                'pacsun': 'pacsun',
                'tjmaxx': 'tjmaxx', 't j maxx': 'tjmaxx', 'tj maxx': 'tjmaxx',
                'toys r us': 'toysrus',
                'lush us': 'lush', 'lush upper west': 'lush',
                'best buy': 'bestbuy',
                'steamgames.com': 'steam',
                'jamba juice': 'jambajuice',
                'jimmy johns': 'jimmyjohns',
                'buffalo wild':'buffalowildwings',
                'tradinoi':'traderjoes',
                 '10belowicecream':'10belowicecream', #START OF JK DATA DUMP
                 '1800contacts': '1800contacts',
                 '99centonly': '99centonly',
                 'lowes': 'lowes',
                 'academysports': 'academysports',
                 'adidas': 'adidas',
                 'aeropostale': 'aeropostale',
                 'albertsons': 'albertsons',
                 'alcatrazisland': 'alcatrazisland',
                 'aldi': 'aldi',
                 'alidoro': 'alidoro',
                 'aliexpress': 'aliexpress',
                 'allchocolatekitchen': 'allchocolatekitchen',
                 'alterdstate': 'alterdstate',
                 'amazon': 'amazon',
                 'amc': 'amc',
                 'americanapparel': 'americanapparel',
                 'americaneagle': 'americaneagle',
                 'amtrak': 'amtrak',
                 'andofood': 'andofood',
                 'anfora': 'anfora',
                 'apple': 'apple',
                 'applebees': 'applebees',
                 'appleitunes': 'appleitunes',
                 'aramark': 'aramark',
                 'arbys': 'arbys',
                 'arcaffe': 'arcaffe',
                 'arco': 'arco',
                 'aromajoes': 'aromajoes',
                 'artichokepizza': 'artichokepizza',
                 'artistandcraftsmansupplies': 'artistandcraftsmansupplies',
                 'astorplatecafe': 'astorplatecafe',
                 'atm': 'atm',
                 'auntieannes': 'auntieannes',
                 'auzaatar': 'auzaatar',
                 'bagelpub': 'bagelpub',
                 'bakedbymelissa': 'bakedbymelissa',
                 'bamrosecinema': 'bamrosecinema',
                 'bandcampmrcarmack': 'bandcampmrcarmack',
                 'baohaus': 'baohaus',
                 'barnesandnoble': 'barnesandnoble','barnesnoble':'barnesandnoble', #JK add
                 'basicsplus': 'basicsplus',
                 'baskinrobbins': 'baskinrobbins','baskin': 'baskinrobbins',#JK add
                 'bathandbodyworks': 'bathandbodyworks',
                 'batteryplacemarket': 'batteryplacemarket',
                 'bbarandgrill': 'bbarandgrill',
                 'beansandgreens': 'beansandgreens',
                 'benandjerrys': 'benandjerrys',
                 'benemon': 'benemon',
                 'bestbuy': 'bestbuy',
                 'bestlockers': 'bestlockers',
                 'beverlys': 'beverlys',
                 'bflat': 'bflat',
                 'biggayicecream': 'biggayicecream',
                 'biglots': 'biglots',
                 'bitecafe': 'bitecafe',
                 'bjsrestaurant': 'bjsrestaurant',
                 'bkbagel': 'bkbagel',
                 'blickartsupplies': 'blickartsupplies',
                 'blizzard': 'blizzard',
                 'bloomingdales': 'bloomingdales',
                 'bluebottlecoffee': 'bluebottlecoffee',
                 'blueribbonfriedchicken': 'blueribbonfriedchicken',
                 'bluestonelane': 'bluestonelane',
                 'bodybuilding': 'bodybuilding',
                 'bonfirebar': 'bonfirebar',
                 'booksamillion': 'booksamillion',
                 'boomerangtoys': 'boomerangtoys',
                 'bornthai': 'bornthai',
                 'bp': 'bp',
                 'brandymelville': 'brandymelville',
                 'brooklyncommunitybookstore': 'brooklyncommunitybookstore',
                 'brothersmarketplace': 'brothersmarketplace',
                 'bucees': 'bucees',
                 'buffalowildwings': 'buffalowildwings',
                 'buildabear': 'buildabear',
                 'burgerking': 'burgerking',
                 'burlington': 'burlington',
                 'butterfieldmarket': 'butterfieldmarket',
                 'cafeanello': 'cafeanello',
                 'cafeghia': 'cafeghia',
                 'cafegrumpy': 'cafegrumpy',
                 'cafelix': 'cafelix',
                 'cafezaiya': 'cafezaiya',
                 'calvinklein': 'calvinklein',
                 'cariboucoffee': 'cariboucoffee',
                 'carlosbakery': 'carlosbakery',
                 'carters': 'carters',
                 'carvelicecream': 'carvelicecream',
                 'caseys': 'caseys',
                 'cavagrill': 'cavagrill',
                 'caviar': 'caviar',
                 'caviarchavela': 'caviarchavela',
                 'caviarcomharryan': 'caviarcomharryan',
                 'caviarspeedyr': 'caviarspeedyr',
                 'caviartinysg': 'caviartinysg',
                 'cbs': 'cbs',
                 'centralparkzoo': 'centralparkzoo',
                 'centurytheatres': 'centurytheatres',
                 'chalkpointkitchen': 'chalkpointkitchen',
                 'championcoffee': 'championcoffee',
                 'champssports': 'champssports',
                 'charitywater': 'charitywater',
                 'charlotterusse': 'charlotterusse',
                 'checkersburgers': 'checkersburgers',
                 'cheeseboy': 'cheeseboy',
                 'chevron': 'chevron',
                 'chickfila': 'chickfila',
                 'chickpea': 'chickpea',
                 'chilis': 'chilis',
                 'chipotle': 'chipotle',
                 'chopt': 'chopt',
                 'cinemark': 'cinemark',
                 'cinnabon': 'cinnabon',
                 'circlek': 'circlek',
                 'citgo': 'citgo',
                 'citylimitsdiner': 'citylimitsdiner',
                 'claires': 'claires',
                 'claudette': 'claudette',
                 'cocacola': 'cocacola',
                 'cocoandcru': 'cocoandcru',
                 'codecademy': 'codecademy',
                 'coffeeproject': 'coffeeproject',
                 'coffepayerets': 'coffepayerets',
                 'coinbase': 'coinbase',
                 'coldstone': 'coldstone',
                 'containerstore': 'containerstore',
                 'costco': 'costco',
                 'costplusworld': 'costplusworld',
                 'crackerbarrel': 'crackerbarrel',
                 'crunchgym': 'crunchgym',
                 'culvers': 'culvers',
                 'cumberlandfarms': 'cumberlandfarms',
                 'curiositystream': 'curiositystream',
                 'current': 'current',
                 'cvs': 'cvs',
                 'dairyqueen': 'dairyqueen',
                 'danny2cleaners': 'danny2cleaners',
                 'daveandbusters': 'daveandbusters',
                 'dd/br': 'dd/br',
                 'deananddeluca': 'deananddeluca',
                 'demarestfarms': 'demarestfarms',
                 'dennys': 'dennys',
                 'dickssportinggoods': 'dickssportinggoods',
                 'diginn': 'diginn',
                 'disneystore': 'disneystore',
                 'dnynaturalland': 'dnynaturalland',
                 'dollargeneral': 'dollargeneral',
                 'dollartree': 'dollartree',
                 'dominos': 'dominos',
                 'dostorostaqueria': 'dostorostaqueria',
                 'doubleknot': 'doubleknot',
                 'doughnutdilemma': 'doughnutdilemma',
                 'dropbox': 'dropbox',
                 'dsquared': 'dsquared',
                 'dsw': 'dsw',
                 'duanereade': 'duanereade',
                 'dunkindonuts': 'dunkindonuts',
                 'eataly': 'eataly',
                 'ebay': 'ebay',
                 'einsteinbros': 'einsteinbros',
                 'essen': 'essen',
                 'etsy': 'etsy',
                 'etsycabreraleather': 'etsycabreraleather',
                 'eversonroycebar': 'eversonroycebar',
                 'evo': 'evo',
                 'extrafancy': 'extrafancy',
                 'facebook': 'facebook',
                 'fairmontmiramarhotel': 'fairmontmiramarhotel',
                 'fairwaymarket': 'fairwaymarket',
                 'familydollar': 'familydollar',
                 'famousfamiglia': 'famousfamiglia',
                 'famousfootwear': 'famousfootwear',
                 'fattydaddytaco': 'fattydaddytaco',
                 'fedex': 'fedex',
                 'felixcontinentalcafe': 'felixcontinentalcafe',
                 'fellowbarber': 'fellowbarber',
                 'financierpatisserie': 'financierpatisserie',
                 'finishline': 'finishline',
                 'firehousesubs': 'firehousesubs',
                 'fivebelow': 'fivebelow',
                 'fiveguys': 'fiveguys',
                 'flatironwinesspirits': 'flatironwinesspirits',
                 'foodlion': 'foodlion',
                 'foodtown': 'foodtown',
                 'footlocker': 'footlocker',
                 'forever21': 'forever21',
                 'francescas': 'francescas',
                 'freemansrestaurant': 'freemansrestaurant',
                 'freepeople': 'freepeople',
                 'freshandco': 'freshandco',
                 'friedmanshomeimprovement': 'friedmanshomeimprovement',
                 'frysfood': 'frysfood',
                 'g2a': 'g2a',
                 'gamestop': 'gamestop',
                 'gardenofeden': 'gardenofeden',
                 'gardnerscandies': 'gardnerscandies',
                 'gasolinealleycoffee': 'gasolinealleycoffee',
                 'geido': 'geido',
                 'giant': 'giant',
                 'glamsquad': 'glamsquad',
                 'godaddy': 'godaddy',
                 'godiva': 'godiva',
                 'goldmansachscanteen': 'goldmansachscanteen',
                 'goodwill': 'goodwill',
                 #'google': 'google', - confusing with google play
                 'googleplay': 'googleplay',
                 'googleplayarturyusupov': 'googleplayarturyusupov',
                 'googleplayciaostudio': 'googleplayciaostudio',
                 'googleplaymixvibes': 'googleplaymixvibes',
                 'googleplaymusically': 'googleplaymusically',
                 'googleplaynoodlecake': 'googleplaynoodlecake',
                 'googleplayoriondroids': 'googleplayoriondroids',
                 'gorillacoffee': 'gorillacoffee',
                 'grainstore': 'grainstore',
                 'gristedes': 'gristedes',
                 'groupon': 'groupon',
                 'grubhub': 'grubhub',
                 'gulf': 'gulf',
                 'gyukaku': 'gyukaku',
                 'halfpricebooks': 'halfpricebooks',
                 'handm': 'handm',
                 'harbs': 'harbs',
                 'harkinstheatres': 'harkinstheatres',
                 'hawkers': 'hawkers',
                 'heb': 'heb',
                 'hersheypark': 'hersheypark',
                 'hobbylobby': 'hobbylobby',
                 'holidaystationstores': 'holidaystationstores',
                 'hollister': 'hollister',
                 'homedepot': 'homedepot',
                 'homespunfoods': 'homespunfoods',
                 'honeybrains': 'honeybrains',
                 'hotelcoronado': 'hotelcoronado',
                 'hottopic': 'hottopic',
                 'houseofyes': 'houseofyes',
                 'hudsonnews': 'hudsonnews',
                 'hudsonsquarepharmacy': 'hudsonsquarepharmacy',
                 'humblebundle': 'humblebundle',
                 'hungryghost': 'hungryghost',
                 'icbrooklyncafe': 'icbrooklyncafe',
                 'icing': 'icing',
                 'igabondisydney': 'igabondisydney',
                 'ihop': 'ihop',
                 'ilbucoalimentarivineria': 'ilbucoalimentarivineria',
                 'inandoutburger': 'inandoutburger',
                 'inmotionentertainment': 'inmotionentertainment',
                 'insomniacookies': 'insomniacookies',
                 'intelligentsiacoffee': 'intelligentsiacoffee',
                 'itssugar': 'itssugar',
                 'jackinthebox': 'jackinthebox',
                 'jacquestorreschocolate': 'jacquestorreschocolate',
                 'jambajuice': 'jambajuice',
                 'jcpenney': 'jcpenney',
                 'jerseymikes': 'jerseymikes',
                 'jimmyjohns': 'jimmyjohns',
                 'joann': 'joann',
                 'joeandthejuice': 'joeandthejuice',
                 'joejrrestaurant': 'joejrrestaurant',
                 'jollywasher': 'jollywasher',
                 'josephleonard': 'josephleonard',
                 'joyyeenoodle': 'joyyeenoodle',
                 'juicegeneration': 'juicegeneration',
                 'juicepress': 'juicepress',
                 'justice': 'justice',
                 'justsalad': 'justsalad',
                 'kaffe1668': 'kaffe1668',
                 'keflavikairportcafe': 'keflavikairportcafe',
                 'keyfood': 'keyfood',
                 'kfc': 'kfc',
                 'khimsmilleniummarket': 'khimsmilleniummarket',
                 'kimchigrill': 'kimchigrill',
                 'kindle': 'kindle',
                 'kingsoopers': 'kingsoopers',
                 'kingtaco': 'kingtaco',
                 'kmart': 'kmart',
                 'koffeecakecorner': 'koffeecakecorner',
                 'kohls': 'kohls',
                 'korilla': 'korilla',
                 'kotobujirestaurant': 'kotobujirestaurant',
                 'krispykreme': 'krispykreme',
                 'kroger': 'kroger',
                 'kulialma': 'kulialma',
                 'kwiktrip': 'kwiktrip',
                 'lacolombe': 'lacolombe',
                 'lafayette': 'lafayette',
                 'laitaliankitchen': 'laitaliankitchen',
                 'lasalette': 'lasalette',
                 'lasolastaqueria': 'lasolastaqueria',
                 'laughingman': 'laughingman',
                 'leancrust': 'leancrust',
                 'lebasket': 'lebasket',
                 'legolandcastleicecream': 'legolandcastleicecream',
                 'lemonade': 'lemonade',
                 'lepainquotidien': 'lepainquotidien',
                 'liquiteria': 'liquiteria',
                 'littlecaesars': 'littlecaesars',
                 'littlemo': 'littlemo',
                 'littleskips': 'littleskips',
                 'lolliandpops': 'lolliandpops',
                 'lordandtaylor': 'lordandtaylor',
                 'luckybrand': 'luckybrand',
                 'luckysupermarket': 'luckysupermarket',
                 'lululemon': 'lululemon',
                 'lunacafeamsterdam': 'lunacafeamsterdam',
                 'lunaparkconeyisland': 'lunaparkconeyisland',
                 'lush': 'lush',
                 'lyft': 'lyft',
                 'mablessmokehouse': 'mablessmokehouse',
                 'macys': 'macys',
                 'madewell': 'madewell',
                 'magnoliabakery': 'magnoliabakery',
                 'maharlika': 'maharlika',
                 'makesandwich': 'makesandwich',
                 'malaproject': 'malaproject',
                 'maman': 'maman',
                 'mamounsfalafel': 'mamounsfalafel',
                 'maple': 'maple',
                 'marshalls': 'marshalls',
                 'maryspizzashack': 'maryspizzashack',
                 'mcdonalds': 'mcdonalds',
                 'meandollies': 'meandollies',
                 'meijer': 'meijer',
                 'menchiesyogurt': 'menchiesyogurt',
                 'michaels': 'michaels',
                 'microsoft': 'microsoft',
                 'milkbar': 'milkbar',
                 'milkmadeicecream': 'milkmadeicecream',
                 'minecraft': 'minecraft',
                 'miscelanea': 'miscelanea',
                 'missionchinese': 'missionchinese',
                 'momofukussambar': 'momofukussambar',
                 'monumentlane': 'monumentlane',
                 'morningtomidnight': 'morningtomidnight',
                 'mortonwilliams': 'mortonwilliams',
                 'mtametronorth': 'mtametronorth',
                 'mtasubway': 'mtasubway',
                 'muji': 'muji',
                 'museumofnaturalhistory': 'museumofnaturalhistory',
                 'museumofscienceboston': 'museumofscienceboston',
                 'myersofkeswick': 'myersofkeswick',
                 'nathans': 'nathans',
                 'nayax': 'nayax',
                 'necessaryclothing': 'necessaryclothing',
                 'netflix': 'netflix',
                 'newburycomics': 'newburycomics',
                 'newjerseytransit': 'newjerseytransit',
                 'nike': 'nike',
                 'nintendo': 'nintendo',
                 'noodlesandcompany': 'noodlesandcompany',
                 'nordstrom': 'nordstrom',
                 'nyctaxi': 'nyctaxi',
                 'officedepot': 'officedepot',
                 'oldnavy': 'oldnavy',
                 'oldtownbar': 'oldtownbar',
                 'olives': 'olives',
                 'onepassword': 'onepassword',
                 'overtheight': 'overtheight',
                 'pacsun': 'pacsun',
                 'pandaexpress': 'pandaexpress',
                 'panerabread': 'panerabread',
                 'panya': 'panya',
                 'papajohns': 'papajohns',
                 'papersource': 'papersource',
                 'parchment': 'parchment',
                 'parking': 'parking',
                 'parrishartmuseum': 'parrishartmuseum',
                 'paylessshoesource': 'paylessshoesource',
                 'paypal': 'paypal',
                 'peetscoffee': 'peetscoffee',
                 'pennmedicine': 'pennmedicine',
                 'pennyfarthing': 'pennyfarthing',
                 'petco': 'petco',
                 'petestavern': 'petestavern',
                 'petfoodexpress': 'petfoodexpress',
                 'petsmart': 'petsmart',
                 'pinkberry': 'pinkberry',
                 'pirate101': 'pirate101',
                 'pizzahut': 'pizzahut',
                 'playstation': 'playstation',
                 'plazatequilla': 'plazatequilla',
                 'plowsharescoffee': 'plowsharescoffee',
                 'popeyes': 'popeyes',
                 'porkbun': 'porkbun',
                 'portillos': 'portillos',
                 'postmates': 'postmates',
                 'potbelly': 'potbelly',
                 'premierfoodfreshmarket': 'premierfoodfreshmarket',
                 'pressbrothersjuicery': 'pressbrothersjuicery',
                 'pressedjuicery': 'pressedjuicery',
                 'pretamanger': 'pretamanger',
                 'prodigycoffee': 'prodigycoffee',
                 'publixsupermarket': 'publixsupermarket',
                 'quansushi': 'quansushi',
                 #'quiktrip': 'quiktrip', -JK Removed hurting atm not getting ang quiktrip
                 'racetrac': 'racetrac',
                 'raisingcanes': 'raisingcanes',
                 'ralphs': 'ralphs',
                 'ramensetagaya': 'ramensetagaya',
                 'randdfoods': 'randdfoods',
                 'rapha': 'rapha',
                 'rayspizzabagel': 'rayspizzabagel',
                 'regalcinemas': 'regalcinemas',
                 'reynardsrestaurant': 'reynardsrestaurant',
                 'riotgames': 'riotgames',
                 'riteaidpharmacy': 'riteaidpharmacy',
                 'rockarollabar': 'rockarollabar',
                 'romwe': 'romwe',
                 'rossstores': 'rossstores',
                 'royalpalmsshuffleboard': 'royalpalmsshuffleboard',
                 'rue21': 'rue21',
                 'safeway': 'safeway',
                 'sallybeauty': 'sallybeauty',
                 'samsclub': 'samsclub',
                 'samuelclemenshighschool': 'samuelclemenshighschool',
                 'samuraipapa': 'samuraipapa',
                 'sarkujapan': 'sarkujapan',
                 'saturdayssurf': 'saturdayssurf',
                 'semsom': 'semsom',
                 'senorpollo': 'senorpollo',
                 'sephora': 'sephora',
                 'seveneleven': 'seveneleven',
                 'shakeshack': 'shakeshack',
                 'sheetz': 'sheetz',
                 'shell': 'shell',
                 'shisomodernasian': 'shisomodernasian',
                 'shoebox': 'shoebox',
                 'shopperspharmacy': 'shopperspharmacy',
                 'shoprite': 'shoprite',
                 'shutterfly': 'shutterfly',
                 'siggysgoodfood': 'siggysgoodfood',
                 'sixflags': 'sixflags',
                 'skdelimarket': 'skdelimarket',
                 'skyzone': 'skyzone',
                 'smashburger': 'smashburger',
                 'smiths': 'smiths',
                 'smithstbagel': 'smithstbagel',
                 'smoothieking': 'smoothieking',
                 'snapchatspectacles': 'snapchatspectacles',
                 'sobaya': 'sobaya',
                 'sockerbit': 'sockerbit',
                 'sonic': 'sonic',
                 'sonomamarket': 'sonomamarket',
                 'specialtys': 'specialtys',
                 'speedway': 'speedway',
                 'spinx': 'spinx',
                 'sportclips': 'sportclips',
                 'sportsmanswarehouse': 'sportsmanswarehouse',
                 'spotify': 'spotify',
                 'sprinklescupcake': 'sprinklescupcake',
                 'sprinklesicecream': 'sprinklesicecream',
                 'sproutsfarmersmarket': 'sproutsfarmersmarket',
                 'sqirl': 'sqirl',
                 'squarecash': 'squarecash',
                 'squarespace': 'squarespace',
                 'staples': 'staples',
                 'starbucks': 'starbucks',
                 'starstable': 'starstable',
                 'steaknshake': 'steaknshake',
                 'steam': 'steam',
                 'stillwellsicecream': 'stillwellsicecream',
                 'stonepercussion': 'stonepercussion',
                 'stripe': 'stripe',
                 'subway': 'subway',
                 'sunoco': 'sunoco',
                 'sunrisemart': 'sunrisemart',
                 'superamerica': 'superamerica',
                 'supremenewyork': 'supremenewyork',
                 'sweetgreen': 'sweetgreen',
                 'tabataramen': 'tabataramen',
                 'tableonten': 'tableonten',
                 'tacobell': 'tacobell',
                 'tacombi': 'tacombi',
                 'tapeo29': 'tapeo29',
                 'taqueriadiana': 'taqueriadiana',
                 'target': 'target',
                 'teaandsymphony': 'teaandsymphony',
                 'tgifridays': 'tgifridays',
                 'thebean': 'thebean',
                 'thebluebellcafe': 'thebluebellcafe',
                 'thebutchersdaughter': 'thebutchersdaughter',
                 'thecheesecakefactory': 'thecheesecakefactory',
                 'thecountingroom': 'thecountingroom',
                 'theellington': 'theellington',
                 'thefoodemporium': 'thefoodemporium',
                 'thefriendlytoast': 'thefriendlytoast',
                 'thegarret': 'thegarret',
                 'thegibson': 'thegibson',
                 'thegreenbean': 'thegreenbean',
                 'thelastbookstore': 'thelastbookstore',
                 'themeltingpot': 'themeltingpot',
                 'thepaperstore': 'thepaperstore',
                 'thepark': 'thepark',
                 'thepokespot': 'thepokespot',
                 'theprince': 'theprince',
                 'thestandard': 'thestandard',
                 'thetang': 'thetang',
                 'thetophatter': 'thetophatter',
                 'thetowncommon': 'thetowncommon',
                 'theupsstore': 'theupsstore',
                 'thewing': 'thewing',
                 'thinkcoffee': 'thinkcoffee',
                 'thirdrailcoffee': 'thirdrailcoffee',
                 'ticketweb': 'ticketweb',
                 'tile': 'tile',
                 'tillys': 'tillys',
                 'timhortons': 'timhortons',
                 'tinycupids': 'tinycupids',
                 'tjmaxx': 'tjmaxx',
                 'tomthumb': 'tomthumb',
                 'tomys': 'tomys',
                 'toysrus': 'toysrus',
                 'traderjoe': 'traderjoe',
                 'tropicalsmoothiecafe': 'tropicalsmoothiecafe',
                 'tswirlcrepe': 'tswirlcrepe',
                 'uber': 'uber',
                 'ultabeauty': 'ultabeauty',
                 'underarmour': 'underarmour',
                 'unionfare': 'unionfare',
                 'unionmarket': 'unionmarket',
                 'uniqlo': 'uniqlo',
                 'urbanoutfitters': 'urbanoutfitters',
                 'usps': 'usps',
                 'uspsmoversguide': 'uspsmoversguide',
                 'utrechtartsupplies': 'utrechtartsupplies',
                 'valleyofthemoonveterinary': 'valleyofthemoonveterinary',
                 'vanessasdumplings': 'vanessasdumplings',
                 'vans': 'vans',
                 'veksler': 'veksler',
                 'vendingmachine': 'vendingmachine',
                 'veselka': 'veselka',
                 'victoriassecret': 'victoriassecret',
                 'vividphoto': 'vividphoto',
                 'vons': 'vons',
                 'wafflehouse': 'wafflehouse',
                 'walgreens': 'walgreens',
                 'walmart': 'walmart',
                 'waltdisneyworld': 'waltdisneyworld',
                 'wawa': 'wawa',
                 'wegmans': 'wegmans',
                 'wendys': 'wendys',
                 'westsidemarket': 'westsidemarket',
                 'wetzelspretzels': 'wetzelspretzels',
                 'whataburger': 'whataburger',
                 'wholefoods': 'wholefoods',
                 'wikipedia': 'wikipedia',
                 'wilmajean': 'wilmajean',
                 'wincofoods': 'wincofoods',
                 'winndixie': 'winndixie',
                 'wish': 'wish',
                 'woopsbakeshop': 'woopsbakeshop',
                 #'xbox': 'xbox',  -Microsoft and Xbox are both used for very simular mercehant strings
                 'xianfamousfoods': 'xianfamousfoods',
                 'yasotangbao': 'yasotangbao',
                 'yiwanmen': 'yiwanmen',
                 'ynbar': 'ynbar',
                 'yogavida': 'yogavida',
                 'yogaworks': 'yogaworks',
                 'yogurtland': 'yogurtland',
                 'youtube': 'youtube',
                 'yucabar': 'yucabar',
                 'zabbelee': 'zabbelee',
                 'zaful': 'zaful',
                 'zagg': 'zagg',
                 'zappos': 'zappos',
                 'zaxbys': 'zaxbys',
                 'zipcar': 'zipcar',
                 'zumiez': 'zumiez',
                 'greatclips':'greatclips' #NEW BRAND
               }

def mapping_dict_funct(df, col_origin, col_output, mapping_dict):
    for key, value in mapping_dict.items():
        df[col_output] = np.where(df[col_origin].str.contains(key), value, df[col_output])

## Map by MCC as well
#mcc_dict = {'7523.0': 'parking', '6011.0': 'atm', '6010.0': 'atm'}
mcc_dict = {'7523': 'parking', '6011': 'atm', '6010': 'atm'} # make sure you've stripped ".0" from mcc already

def mcc_dict_funct(df, col_origin, col_output, mcc_dict):
    for key, value in mcc_dict.items():
        df[col_output] = np.where(df[col_origin]==key, value, df[col_output]) # (df[col_origin].notna()) & (

# Run dictionary functions on data
mapping_dict_funct(data, 'merchant_string_dict', 'mapped_brand_dict_3', mapping_dict)
mcc_dict_funct(data, 'mcc', 'mapped_brand_dict_3', mcc_dict)

# Create new column flags for predicted, correct, train/test, row count
# Can be used to summarize, if desired
data.fillna('None', inplace=True)
data['predicted_flag'] = np.where(data['mapped_brand_dict_3'] != 'None',1,0)
data['equals_mb_flag_1'] = np.where(data['mapped_brand'] == data['mapped_brand_dict_3'],1,0)
data['train_test'] = np.where(data['mapped_brand'].notna(),'train','test')
data['one_per_row'] = 1


## <span style="color:Orange"> **Write .csv to wd with labeled data** </span>

In [6]:
data[data['predicted_flag']==1].to_csv('dictionary_labels_DNsssE.csv')